### Agent
The Agent implementation can be found in the root directory as the class Agent in the file _agent_single_actor_dinkyper.py_.

The agent can have many instances of itself which all communicate to the same D4PG "brain" in order to make decisions and provide its experiences to.  The agent has 2 functions that communicate with the brain: act() which gets the action vector based on the agent's state as in input to the actor network, and step() which simply adds the acted out experience of the agent to the D4PG's memory buffer.

The underlying D4PG brain utilized the following hyperparameters for the learning process:
## OUNoise hyperparams
- mu = 0
- theta = 0.075 #0.15
- sigma = 0.075 #0.2

## Memory hyperparams
- buffer_size = 100000
- batch_size = 16
- good_memories_enabled = True
- good_memory_batch_representation = 0.25
- good_memory_experience_lead = 20

## Update hyperparams
- gamma = 0.99
- tau = 0.01
- alr = 1e-4
- clr = 1e-3

The learning algorithm works as follows:
1. Each agent gets the action vector that it should perform from the local actor network giving the agent's state as input.  Some Ornstein Uhlenbeck noise is also added to the action vector in order to introduce some small level of exploration but not in an erratic but more of a consistent and predictable amount of random noise.  This noise added continuous action is clipped at -1 and 1 so that the noise doesn't accidentally put the action past the expected bounds.
2. Both agents take a step in the environment using the action vector from step 1 and observes the reward, next state, and done status and feeds all of this information, as well as the combined current and next states of both agents, in to one memory buffer shared by both agents.
3. After every time step a small batch from the buffer is used for learning for the actor and critic networks.  The memory buffer has a standard queue that samples most of its memories from for each sampling operation, but it also utilizes a "good memory" queue.  This separate queue is the same size as the primary queue but at every sampling step a fraction of the samples are taken from the "good memory queue" controlled by the hyperparameter *good_memory_batch_representation* and the rest are taken from the regular queue.  In order to be added to the "good memory queue", the would-be sample has to have a reward that is better or equal to the average reward in that queue.  In order to prevent overfitting and overusing the experiences from the good memory queue, every time the regular memory queue does a "full refresh" (memorynum % buffer_size == 0) that regular memory will be added to the "good memory queue".  Additionally, since these "high rewarding experiences" are out of context while alone, a recent sampling of experiences leading to this good experience are added to the "good memory queue" which is controlled by the hyperparameter *good_memory_experience_lead*.
4. A) The Actor learns by utilizing the critic's opinion of how the actor chose an action from the state with the added context of the other agent's state and chosen action.  More formally, it takes a step in the direction of maximizing Critic(full_state, [Actor(partial_state) for Actor in all_actors]) where Critic gives the value of the complete state from both agents perspectives and the actions of each agent.
4. B) The Critic learns by comparing its value evaluation of the current full state and both agent's actions to the observed reward and discounted(gamma) value of the next full state and the actions that the actors would take from that state under the current policy.  Both of these learning steps appropriately use target networks so that learning is more stable and updates these target networks based on the hyperparameter tau.

### Actor and Critic models

#### Actor
The actor model is found as the class Actor in the file agent.py.  The model consists of 3 hidden layers: 
- fully connected, size 32, relu activation
- fully connected, size 64, relu activation
- fully connected, size 64, relu activation

The output layer has a tanh activation function in order to bound the action vector's values between -1 and 1.

The hidden layers are also initialized with Kaiming (He) uniform weight distributions and the output layer is initialized by Xavier uniform weight distribution.  This is done because layers with ReLu activations learn faster and more effectively from Kaiming weight distributions, and layers with tanh or other logistical activations learn more effectively from Xavier weight distributions.

#### Critic
The critic model is found as the class Critic in the file agent.py.  The model combines the state and action vector as one tensor and uses that as the input.  There are 2 hidden layers in this network:
- fully connected, size 64, relu activation
- fully connected, size 64, relu activation

The output layer uses the natural output in order to give the normal, unbounded value of the inputs.

All hidden layers and the output layer are initialized with Kaiming uniform weight distributions.

### Rewards plot
Below is the reward plot from taking the max score from both agents per episode.  This project by far pushed me the most and made me try several techniques, including custom ones of my own, and training never seemed like it was actually learning but just "getting it" or "not getting it".  This is pretty apparent by the graph below where the score is very tine and then shoots up but does not remain consistent.  The only upward trend here is the increased consistency of "good performance".  This could probably be attributed to the fact that both agents are responsible for the score, and much like real life, two great individual players may perform tragically if they are not properly working together.  Eventually, on the 13477th episode, the goal was acheived of reaching an average score over the last 100 episodes of 0.5.

<img src="scores_plot.png">

### Possible Future Improvements

Some implementations I tried but lead to poor results were an actual prioritized experience queue, instead of my *Dinky Prioritized Experience Queue&trade;*, separate actor networks/experience queues for each agent, and flipping the order of state vectors when committing memories of each agent. 

Some possible future improvements that could be made are training the agents in phases, first by getting the agents to just hit the ball to the other side and only consider itself, THEN adding in the "full picture" to the critic.  This can speed up the beginning of the training by having less forward passes and a thinner data set to process.  

One of the problems with these agents were that they were just falling into a weird local minima of going all the way to the net or all the way back and the only reason it trained at all was that the exploration from the noise lead to random encounters with the ball.  Another implementation that could prove to be beneficial is a decaying exploration that starts much higher then lowers in later episodes rather than OU noise that stays at the same rate throughout.